# 🛠 Setup

In [ ]:
pip install -U "servicefoundry==0.6.6" "mlfoundry==0.6.1"

In [1]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [2]:
!sfy login

/Users/nikp18/.truefoundry/profiles.json file is deprecated. You can delete this file now.
/Users/nikp18/.truefoundry/sessions.json file is deprecated. You can delete this file now.
Already logged in to 'https://app.truefoundry.com' as 'nikhil' ('nikhil@truefoundry.com')
Please use `sfy login --relogin` or `sfy.login(relogin=True)` to force relogin



In [3]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass

WORKSPACE="tfy-cluster-euwe1:hardik-ws"
TFY_API_KEY = getpass("Please enter your API Key,( https://app.truefoundry.com/settings ):-")

Please enter your API Key,( https://app.truefoundry.com/settings ):-········


# ⚡ Jobs - Quick Start
Here we will create a simple job that counts to 30 sleeping for 1 second every iteration.

In [4]:
!mkdir -p demo/simple/job
%cd demo/simple/job

/Users/nikp18/work/repos/tempp/truefoundry-examples/end-to-end-examples/demo/simple/job


In [5]:
%%writefile run.py

import argparse
import time


def main(limit: int):
    for i in range(limit):
        print(f"Running Job: {i + 1}")
        time.sleep(1)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--limit", default=10, type=int)
    args = parser.parse_args()
    main(args.limit)

Writing run.py


In [6]:
%%writefile deploy.py

import logging

from servicefoundry import Build, Job, PythonBuild, Resources

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

job = Job(
    name="counter-job",
    image=Build(build_spec=PythonBuild(command="python run.py")),
    resources=Resources(cpu_request=0.5, cpu_limit=1.0)
)
job.deploy(workspace_fqn=WORKSPACE)

Writing deploy.py


# ☁ Service - Quick Start
Here we will take two python functions that use numpy and register them as routes in a deployed web service

In [7]:
%cd ../../..
! mkdir -p demo/simple/service
%cd demo/simple/service

/Users/nikp18/work/repos/tempp/truefoundry-examples/end-to-end-examples
/Users/nikp18/work/repos/tempp/truefoundry-examples/end-to-end-examples/demo/simple/service


In [8]:
%%writefile service.py
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}

Writing service.py


In [9]:
%%writefile requirements.txt
fastapi==0.89.1
uvicorn==0.20.0

Writing requirements.txt


In [10]:
%%writefile deploy.py
import logging

from servicefoundry import Build, Service, PythonBuild, Resources

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

service = Service(
    name="fastapi-service",
    image=Build(build_spec=PythonBuild(command="uvicorn service:app --host 0.0.0.0 --port 8080")),
    resources=Resources(cpu_request=0.5, cpu_limit=1.0),
    ports=[{"port": 8080}]
)
service.deploy(workspace_fqn=WORKSPACE)

Writing deploy.py


# ☁ Function as Service - Quick Start
Here we will take two python functions that use numpy and register them as routes in a deployed web service

In [15]:
# %cd ../../..
! mkdir -p demo/simple/function-service
%cd demo/simple/function-service

/Users/nikp18/work/repos/tempp/truefoundry-examples/end-to-end-examples/demo/simple/function-service


In [16]:
%%writefile functions.py
from typing import List
import numpy as np

def multiply(a: float, b: float):
  return a * b 

def normal(loc: float, scale: float, size: List[int]):
    return np.random.normal(loc=loc, scale=scale, size=size).tolist()

def uniform(low: float, high: float, size: List[int]):
    return np.random.uniform(low=low, high=high, size=size).tolist()

Writing functions.py


In [17]:
%%writefile requirements.txt
numpy==1.21.6

Writing requirements.txt


In [18]:
%%writefile deploy.py
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

from functions import normal, uniform, multiply
from servicefoundry.function_service import FunctionService

service = FunctionService(name="fn-service")
service.register_function(normal)
service.register_function(uniform)
service.register_function(multiply)
deployment = service.deploy(workspace_fqn=WORKSPACE)

Writing deploy.py
